# 生成 AI エージェント

https://docs.cohere.com/v2/docs/building-an-agent-with-cohere

In [ ]:
import logging
import os
#import yaml
import json
import numpy as np
from typing import List
from dotenv import load_dotenv
import cohere

load_dotenv()
co = cohere.ClientV2(api_key=os.environ['COHERE_API_KEY'])

# ツールの作成

ここでは Python 関数で、それぞれのtoolに対応する機能を作成。

In [ ]:
# Create the tools
def search_faqs(query):
    faqs = [
        {
            "text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."
        },
        {
            "text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."
        },
    ]
    return faqs


def search_emails(query):
    emails = [
        {
            "from": "it@co1t.com",
            "to": "david@co1t.com",
            "date": "2024-06-24",
            "subject": "Setting Up Your IT Needs",
            "text": "Greetings! To ensure a seamless start, please refer to the attached comprehensive guide, which will assist you in setting up all your work accounts.",
        },
        {
            "from": "john@co1t.com",
            "to": "david@co1t.com",
            "date": "2024-06-24",
            "subject": "First Week Check-In",
            "text": "Hello! I hope you're settling in well. Let's connect briefly tomorrow to discuss how your first week has been going. Also, make sure to join us for a welcoming lunch this Thursday at noon—it's a great opportunity to get to know your colleagues!",
        },
    ]
    return emails


def create_calendar_event(date: str, time: str, duration: int):
    # You can implement any logic here
    return {
        "is_success": True,
        "message": f"Created a {duration} hour long event at {time} on {date}",
    }


functions_map = {
    "search_faqs": search_faqs,
    "search_emails": search_emails,
    "create_calendar_event": create_calendar_event,
}

ツールの定義

In [ ]:
# Define the tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_faqs",
            "description": "Given a user query, searches a company's frequently asked questions (FAQs) list and returns the most relevant matches to the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query from the user",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_emails",
            "description": "Given a user query, searches a person's emails and returns the most relevant matches to the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query from the user",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "the date on which the event starts, formatted as mm/dd/yy",
                    },
                    "time": {
                        "type": "string",
                        "description": "the time of the event, formatted using 24h military time formatting",
                    },
                    "duration": {
                        "type": "number",
                        "description": "the number of hours the event lasts for",
                    },
                },
                "required": ["date", "time", "duration"],
            },
        },
    },
]

システムを構成する要素:
* ユーザ
* アプリケーション
* LLM
* ツール

処理の手順:
1. ユーザメッセージの取得: LLM がアプリケーションを介してユーザの入力を受け取る。
2. ツール計画と呼び出し: LLM は、ツールを使用することを決定する。そしてツール呼び出しを生成する。
3. ツール実行: アプリケーションは、ツールを実行し、結果を LLM に送る。
4. 応答と引用の生成: LLM は、応答と引用を生成し、ユーザへ返す。

`chat` に `tools` で渡す。

In [ ]:
# Create custom system message
system_message = """## Task and Context
You are an assistant who assist new employees of Co1t with their first week. You respond to their questions and assist them with their needs. Today is Monday, June 24, 2024"""


# Step 1: Get user message
message = "Is there any message about getting setup with IT?"

# Add the system and user messages to the chat history
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": message},
]

# Step 2: Tool planning and calling
response = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)

if response.message.tool_calls:
    print("Tool plan:")
    print(response.message.tool_plan, "\n")
    print("Tool calls:")
    for tc in response.message.tool_calls:
        print(
            f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}"
        )

    # Append tool calling details to the chat history
    messages.append(response.message)


## ツールの実行

`tool` の `['function']['name']` で持たせた名称と、`function_map` のキーが対応していること。

ref. https://docs.cohere.com/v2/docs/tool-use-parameter-types

In [ ]:
# Step 3: Tool execution
for tc in response.message.tool_calls:
    tool_result = functions_map[tc.function.name](
        **json.loads(tc.function.arguments)
    )
    tool_content = []
    for data in tool_result:
        tool_content.append(
            {
                "type": "document",
                "document": {"data": json.dumps(data)},
            }
        )
        # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
    # Append tool results to the chat history
    messages.append(
        {
            "role": "tool",
            "tool_call_id": tc.id,
            "content": tool_content,
        }
    )

    print("Tool results:")
    for result in tool_content:
        print(result)


## 応答と引用の生成

In [ ]:
# Step 4: Response and citation generation
response = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)

# Append assistant response to the chat history
messages.append(
    {"role": "assistant", "content": response.message.content[0].text}
)

# Print final response
print("Response:")
print(response.message.content[0].text)
print("=" * 50)

# Print citations (if any)
if response.message.citations:
    print("\nCITATIONS:")
    for citation in response.message.citations:
        print(citation, "\n")


# マルチステップのツール使用

In [ ]:
model = "command-a-03-2025"

system_message = """## Task and Context
You are an assistant who assists new employees of Co1t with their first week. You respond to their questions and assist them with their needs. Today is Monday, June 24, 2024"""


def run_assistant(query, messages=None):
    if messages is None:
        messages = []

    if "system" not in {m.get("role") for m in messages}:
        messages.append({"role": "system", "content": system_message})

    # Step 1: get user message
    print(f"Question:\n{query}")
    print("=" * 50)

    messages.append({"role": "user", "content": query})

    # Step 2: Generate tool calls (if any)
    response = co.chat(model=model, messages=messages, tools=tools)

    while response.message.tool_calls:

        print("Tool plan:")
        print(response.message.tool_plan, "\n")
        print("Tool calls:")
        for tc in response.message.tool_calls:
            print(
                f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}"
            )
        print("=" * 50)

        messages.append(response.message)

        # Step 3: Get tool results
        for idx, tc in enumerate(response.message.tool_calls):
            tool_result = functions_map[tc.function.name](
                **json.loads(tc.function.arguments)
            )
            tool_content = []
            for data in tool_result:
                tool_content.append(
                    {
                        "type": "document",
                        "document": {"data": json.dumps(data)},
                    }
                )
                # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": tool_content,
                }
            )

        # Step 4: Generate response and citations
        response = co.chat(
            model=model, messages=messages, tools=tools
        )

    messages.append(
        {
            "role": "assistant",
            "content": response.message.content[0].text,
        }
    )

    # Print final response
    print("Response:")
    print(response.message.content[0].text)
    print("=" * 50)

    # Print citations (if any)
    if response.message.citations:
        print("\nCITATIONS:")
        for citation in response.message.citations:
            print(citation, "\n")

    return messages


In [ ]:
messages = run_assistant(
    "Can you check if there are any lunch invites, and for those days, create a one-hour event on my calendar at 12PM."
)
